In [98]:
import pandas as pd
import numpy as np

In [99]:
df = pd.read_csv("matched_data.csv")

In [100]:
#df[['Age','H6','H6.1']]

In [101]:
clean_df = df[:][df['H6'] != 'DK']

In [102]:
clean_df['H6'] = pd.to_numeric(clean_df['H6'])

In [103]:
oldwidth=pd.get_option('display.max_colwidth') # show the full width of SA2 column
pd.set_option('display.max_colwidth', -1)

**Here are the comments from those with negative H6 changes**

In [104]:
clean_df['SA2'][clean_df['H6.1']-clean_df['H6'] < 0]

7     Projects take a lot of work, try something small first                    
13    NaN                                                                       
23    I learned how to use HEAT for OpenStack                                   
26    NaN                                                                       
31    NaN                                                                       
35    If you don’t konw a lot you can still do a lot                            
36    NaN                                                                       
41    Collaboration makes for better projects                                   
45    NaN                                                                       
46    To make sure you have good documentation on any installation/set up steps.
Name: SA2, dtype: object

**Here are the comments fromm those with negative H5 changes**

In [105]:
clean_df['SA2'][clean_df['H5.1']-clean_df['H5'] < 0]

1     Collaboratively/breaking up tasks and asking for help is important.                                                              
5     That most people, including professors and professionals are as lost as me sometimes                                             
22    Learned new terms, got introduction of a few new platforms which I’ll study now on my own. Good mentors and open stack organizers
23    I learned how to use HEAT for OpenStack                                                                                          
26    NaN                                                                                                                              
41    Collaboration makes for better projects                                                                                          
45    NaN                                                                                                                              
Name: SA2, dtype: object

In [106]:
pd.set_option('display.max_colwidth', oldwidth) # set the display width back to the default value

In [107]:
thing = clean_df[['Country','H1Diff','H2Diff','H3Diff','H4Diff','H5Diff','H6Diff']]

In [108]:
thing = thing.replace('#VALUE!', np.nan)

In [109]:
thing = thing.apply(pd.to_numeric, errors='ignore')

**Here are the counts of attendees by country**

In [110]:
thing.groupby('Country').count()

,H1Diff,H2Diff,H3Diff,H4Diff,H5Diff,H6Diff
Country,,,,,,
Bangladesh,1,1,1,1,1,1
Brazil,1,1,1,1,1,1
Canada,1,1,1,1,1,1
China,3,3,3,3,3,3
Germany,2,2,2,2,2,2
Hong Kong,1,1,1,1,1,1
India,13,12,9,10,12,12
New Zealand,1,1,1,1,1,1
Nigeria,1,0,0,0,1,1


**Average change by country**

Here we calculate the average change from pre-to-post for each country. The countries with the largest number of attendees are India, USA and arguable China. I note that all categories for USA and China had an average negative change. For India, the changes were neutral or positive except for H6.

In [111]:
thing.groupby('Country').mean()

,H1Diff,H2Diff,H3Diff,H4Diff,H5Diff,H6Diff
Country,,,,,,
Bangladesh,1.000000,1.000000,2.0000,2.000000,1.000000,2.000000
Brazil,-1.000000,-1.000000,-1.0000,-1.000000,0.000000,-1.000000
Canada,-1.000000,-1.000000,0.0000,0.000000,0.000000,-1.000000
China,-0.666667,0.000000,0.0000,0.000000,-0.333333,-0.333333
Germany,0.000000,0.000000,0.5000,0.000000,-0.500000,1.500000
Hong Kong,1.000000,1.000000,0.0000,1.000000,1.000000,1.000000
India,0.000000,0.250000,0.0000,0.200000,0.000000,-0.166667
New Zealand,0.000000,0.000000,0.0000,-5.000000,-5.000000,-4.000000
Nigeria,0.000000,NaN,NaN,NaN,0.000000,0.000000
